In [1]:
%pip install utils
%pip install scikit-learn
%pip install contractions
%pip install Sastrawi
%pip install swifter
%pip install emoji
import pandas as pd
import utils
import random
import numpy as np
import emoji
import contractions
import re
import string
from sklearn.naive_bayes import MultinomialNB
from scipy.sparse import lil_matrix
from sklearn.feature_extraction.text import TfidfTransformer

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [56]:
import zipfile

# def extract_zip(zip_file, extract_dir):
#   with zipfile.ZipFile(zip_file, 'r') as zip_ref:
#       zip_ref.extractall(extract_dir)

# Contoh penggunaan:
# zip_file_path = "/Users/alipmaskhuri/skripsi/ulasankominfo.zip"
# extract_path = "/Users/alipmaskhuri/skripsi" 
# extract_zip(zip_file_path, extract_path)

file_path = "/Users/alipmaskhuri/skripsi/Dataset/dataset.csv"
df = pd.read_csv(file_path)

In [57]:
df.shape

(17479, 1)

In [35]:
df

,komentar
0,@alimheng @jntexpressid @tokopedia @TokopediaC...
1,@intinyadeh Udah banyak bgt korban @tokopedia ...
2,@islaminoiano07 @tokopedia @JNE_ID @TokopediaC...
3,@islaminoiano07 @tokopedia @JNE_ID @TokopediaC...
4,@islaminoiano07 @tokopedia @JNE_ID @TokopediaC...
...,...
17474,tadi pernah kasi bintang 5 update akhir buat t...
17475,tempat belanja favorit saran aja buat tokopedi...
17476,apabila aplikasi update baru pasti berat jalan...
17477,tampil baru sekarang sangat bagustapi rincian ...


**Menghapus Kolom yang tidak perlu**

In [7]:
# Menghapus kolom yang diinginkan
df = df.drop(columns=['conversation_id_str', 'created_at', 'favorite_count', 'id_str', 'image_url', 'in_reply_to_screen_name', 'lang', 'location', 'quote_count', 'reply_count', 'retweet_count', 'tweet_url', 'user_id_str', 'username'])
df.head(20)


,full_text
0,@alimheng @jntexpressid @tokopedia @TokopediaC...
1,@intinyadeh Udah banyak bgt korban @tokopedia ...
2,@islaminoiano07 @tokopedia @JNE_ID @TokopediaC...
3,@islaminoiano07 @tokopedia @JNE_ID @TokopediaC...
4,@islaminoiano07 @tokopedia @JNE_ID @TokopediaC...
5,@islaminoiano07 @tokopedia @JNE_ID @TokopediaC...
6,@islaminoiano07 @tokopedia @JNE_ID @TokopediaC...
7,@islaminoiano07 @tokopedia @JNE_ID @TokopediaC...
8,@islaminoiano07 @tokopedia @JNE_ID @TokopediaC...
9,@islaminoiano07 @tokopedia @JNE_ID @TokopediaC...


In [58]:
df = df.rename(columns={'full_text': 'komentar'})

In [59]:
print(df['komentar'])

0        @alimheng @jntexpressid @tokopedia @TokopediaC...
1        @intinyadeh Udah banyak bgt korban @tokopedia ...
2        @islaminoiano07 @tokopedia @JNE_ID @TokopediaC...
3        @islaminoiano07 @tokopedia @JNE_ID @TokopediaC...
4        @islaminoiano07 @tokopedia @JNE_ID @TokopediaC...
                               ...                        
17474    tadi pernah kasi bintang 5 update akhir buat t...
17475    tempat belanja favorit saran aja buat tokopedi...
17476    apabila aplikasi update baru pasti berat jalan...
17477    tampil baru sekarang sangat bagustapi rincian ...
17478    lemot nyaa hadeh udah enak pake dulu malah bar...
Name: komentar, Length: 17479, dtype: object


Data Cleaning

In [60]:
def clean_tweet(komentar):
    if not isinstance(komentar, str):  # Jika bukan string, ubah ke string kosong
        komentar = ""
    # Remove RT tags, usernames, URLs, and emojis
    komentar = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", komentar)
    komentar = emoji.replace_emoji(komentar, replace='')

    # Remove extra spaces, hashtags, and punctuation
    komentar = re.sub(r"\s+", " ", komentar)  # Remove extra spaces
    komentar = re.sub(r"#+", "", komentar)  # Remove extra hashtags
    komentar = re.sub(r"[^\w\s]", "", komentar)  # Remove punctuation

    # Convert to lowercase and expand contractions
    komentar = komentar.lower()
    komentar = contractions.fix(komentar)

    # Remove repeated characters
    komentar = re.sub(r"(.)\1+", r"\1\1", komentar)

    return komentar

In [61]:
df['komentar'] = df['komentar'].fillna("").astype(str)

# Terapkan fungsi clean_tweet
df['komentar'] = df['komentar'].apply(clean_tweet)

In [62]:
print(df['komentar'])

0         dlu saya jg mkir gtu sbg pembeli tp trnyata s...
1         udah banyak bgt korban sm kalo ga salah belum...
2         saran saya sebaiknya ambil langkah hukum mela...
3         tolong bantu viralin min kaga beres udh ini b...
4         marketplace harus berani putus kerja sama bar...
                               ...                        
17474    tadi pernah kasi bintang 5 update akhir buat t...
17475    tempat belanja favorit saran aja buat tokopedi...
17476    apabila aplikasi update baru pasti berat jalan...
17477    tampil baru sekarang sangat bagustapi rincian ...
17478    lemot nyaa hadeh udah enak pake dulu malah bar...
Name: komentar, Length: 17479, dtype: object


In [69]:
df.drop_duplicates(subset='komentar',inplace=True)
df.shape

(16487, 1)

In [70]:
df = df.dropna(subset=['komentar'])

In [71]:
df

,komentar
0,dlu saya jg mkir gtu sbg pembeli tp trnyata s...
1,udah banyak bgt korban sm kalo ga salah belum...
2,saran saya sebaiknya ambil langkah hukum mela...
3,tolong bantu viralin min kaga beres udh ini b...
4,marketplace harus berani putus kerja sama bar...
...,...
17474,tadi pernah kasi bintang 5 update akhir buat t...
17475,tempat belanja favorit saran aja buat tokopedi...
17476,apabila aplikasi update baru pasti berat jalan...
17477,tampil baru sekarang sangat bagustapi rincian ...


**Normalisasi**

Menghapus kata yang tidak terdeteksi dalam KBBI

In [72]:
norm = {" dlu " : " dulu ", " dl ":" dulu " , " jg " : " juga " , " mkir " : " mikir " , " sbg " : " sebagai ", " tp" : " tapi ", " bsa" : " bisa ", "serem" : "seram",
        " dpt " : " mendapatkan ", " bgt " : " banget ", " sm " : " sama ", " kaga " : " kagak ", " udh " : " sudah ", " kl " : " kalau ", " ga " : " tidak ", " dpt " : " dapat ",
        " ad " : " ada ", " bkas " : " bekas ", " sus " : " mencurigakan ", " pd " : " pada ",  " bnyk " : " banyak",  " jd " : " jadi ",  " krn " : " karena ",
        " mendng " : " mending ", " smua " : " semua ", " yg " : " yang ",  " org " : " orang ",  " belom " : " belum ",  " nympe " : " sampai ",  " tokped " : " tokopedia ", 
        " klean " : " kalian ", " Cs " : " customer service ", " ppk " : " jelek ", " ga " : " gak ",  " hack " : " retas ", " kpd " : " kepada ",  " tdk " : " tidak ",
        " sender " : " pengirim ",  " karna " : " karena ", " lama2 " : " lama-lama ", " toko ijo " : " tokopedia ", " emng " : " memang ",  " kalo " : " kalau ", " app " : " aplikasi ", 
        " klo " : " kalau ", " notif" : " notifikasi ",  " skrg " : " sekarang ", " apk " : " aplikasi ", " q " : " saya ", " lgi " : " lagi ",  " gmn " : " gimana ",
        " sdh " : " sudah ", " tsb " : " tersebut ",  " sm " : " sama ",  " lg " : " lagi ",  " bln " : " bulan ", " sy " : " saya ", " gua " : " saya ", " prlu ": " perlu ",
        " gw " : " saya ",  " dr " : " dari ",  " grgr " : " gara-gara ",  " tsb " : " tersebut ", " keknya " : " kayaknya ",  " emg " : " memang ",  " uda " : "sudah", " blm " : " belum ",
        " udh " : " sudah ", " VA " : " virtual akun ",  " kyk " : " seperti ",  " dlm " : " dalam ",  " ttg " : " tentang ",  " memesan " : " pesan ", " toped " : " tokopedia ",
        " puas " : " senang ", " gubris " : " merespon ", " jaid ": " jadi ", " liat ": " lihat ",  " btw " : " ngomong ngomong ",  " jngan " : " jangan ", " jngn " : " jangan ",
        " dgn " : " dengan ",   " tlong " : " tolong ", " trnyata " : " ternyata ", " nmr " : " nomor ", " bngt " : " banget " , " bnget " : " banget ", " sdh " :" sudah ",
        " bnyk ": " banyak ", " bnyak " : " banyak " , " yg " : " yang " , " tibatiba " : " tiba-tiba " , " nympe " : " sampai " , " klen " : " kalian ", " kwatir " : " khawatir ",
        " org " : " orang ", " abis " : " habis ", " guys " : " teman ", " guyz " : " teman ", " pd " : " pada ", " ga " : " tidak ", " ekspedisi " : " pengiriman ", " jk " : " jika ",
        " min " : " admin ", " mgkn " : " mungkin ", " lngsng " : " langsung ", " krna " : " karena ", " bntu " : " bantu " , " gue " : " saya ", " gtu " : " begitu ", " bbrp " : " beberapa",
        " kmrn " : " kemarin "}
def normalisasi(str_text):
    for i in norm:
        str_text = str_text.replace(i, norm[i])
    return str_text

df['komentar'] = df['komentar'].apply(lambda x: normalisasi(x))

In [73]:
print(df['komentar'])

0         dulu saya juga mikir begitu sebagai pembeli t...
1         udah banyak banget korban sama kalau tidak sa...
2         saran saya sebaiknya ambil langkah hukum mela...
3         tolong bantu viralin admin kagak beres sudah ...
4         marketplace harus berani putus kerja sama bar...
                               ...                        
17474    tadi pernah kasi bintang 5 update akhir buat t...
17475    tempat belanja favorit saran aja buat tokopedi...
17476    apabila aplikasi update baru pasti berat jalan...
17477    tampil baru sekarang sangat bagustapi rincian ...
17478    lemot nyaa hadeh udah enak pake dulu malah bar...
Name: komentar, Length: 16487, dtype: object


**Stopword Removal**

Mengapus kata yang kurang berbobot atau kurang bermakna

In [74]:
import Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
more_stop_word = []

stop_words = StopWordRemoverFactory().get_stop_words()
new_array = ArrayDictionary(stop_words)
Stop_words_remover_new = StopWordRemover(new_array)

def stopwords(str_text):
    str_text= Stop_words_remover_new.remove(str_text)
    return str_text
df['komentar'] = df['komentar'].apply(lambda x: stopwords(x))

In [75]:
print(df['komentar'])

0         dulu mikir pembeli  ternyata sebagai penjual ...
1         udah banyak banget korban sama kalau salah la...
2         saran sebaiknya ambil langkah hukum melaporka...
3         bantu viralin admin kagak beres ini banyak be...
4         marketplace berani putus kerja sama bareng jn...
                               ...                        
17474    tadi pernah kasi bintang 5 update akhir buat t...
17475    tempat belanja favorit saran aja buat tokopedi...
17476    apabila aplikasi update baru berat jalan fungs...
17477    tampil baru sekarang sangat bagustapi rincian ...
17478    lemot nyaa hadeh udah enak pake dulu malah bar...
Name: komentar, Length: 16487, dtype: object


**Tokenization**

memecah teks menjadi unit-unit yang lebih kecil atau kata-kata individual

In [76]:
tokenized = df['komentar'].apply(lambda x:x.split())
tokenized

0        [dulu, mikir, pembeli, ternyata, sebagai, penj...
1        [udah, banyak, banget, korban, sama, kalau, sa...
2        [saran, sebaiknya, ambil, langkah, hukum, mela...
3        [bantu, viralin, admin, kagak, beres, ini, ban...
4        [marketplace, berani, putus, kerja, sama, bare...
                               ...                        
17474    [tadi, pernah, kasi, bintang, 5, update, akhir...
17475    [tempat, belanja, favorit, saran, aja, buat, t...
17476    [apabila, aplikasi, update, baru, berat, jalan...
17477    [tampil, baru, sekarang, sangat, bagustapi, ri...
17478    [lemot, nyaa, hadeh, udah, enak, pake, dulu, m...
Name: komentar, Length: 16487, dtype: object

**Stemming**

menghilangkan imbuhan (awalan, akhiran, atau infiks) dari sebuah kata dalam bahasa indonesia menggunakan library sastrawi

In [78]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def steming(komentar):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []
    for i in komentar:
        dt = stemmer.stem(i)
        do.append(dt)
    d_clean = []
    d_clean = " ".join(do)
    print(d_clean)
    return d_clean

tokenized = tokenized.apply(steming)

tokenized.to_csv('/Users/alipmaskhuri/skripsi/Dataset/dataset_clean.csv', index=False)
data_clean = pd.read_csv('/Users/alipmaskhuri/skripsi/Dataset/dataset_clean.csv', encoding='latin1')



dulu mikir beli nyata bagai jual jadi korban pas ketemu beli curang youtube ada reviewer jual hp bekas review nya eh beli curang juga jadi semua pihak bisa jadi laku cm jual atau pihak jasa kirim beli jg
udah banyak banget korban sama kalau salah lama ada kasus sama
saran baik ambil langkah hukum lapor polisi jual market place laku konsumen lama korban alami rugi atas 2 5 juta
bantu viralin admin kagak beres ini banyak bener korban nya udh
marketplace berani putus kerja sama bareng jne kalau sistem dibenerin makin banyak korban hey kurir pikir panjang kalau mau buat kriminal kalau putus kerja sama semua marketplace di phk lu ingat percaya no 1
jne rasa rosalia indah curiga banget laporin aja polisi kalau gin mah bikin paguyuban korban jne
habis baca nyata juga jadi korban kirim kamera status gak orang rumah terima chat sama kurir bilang paket ilang
ya kalian tidak kapok begitu banyk kasus cukup kami2 jadi korban harap untung lebih giur pot harga hingga cashback ujung stres sial yang sa

In [79]:
df

,komentar
0,dulu mikir pembeli ternyata sebagai penjual ...
1,udah banyak banget korban sama kalau salah la...
2,saran sebaiknya ambil langkah hukum melaporka...
3,bantu viralin admin kagak beres ini banyak be...
4,marketplace berani putus kerja sama bareng jn...
...,...
17474,tadi pernah kasi bintang 5 update akhir buat t...
17475,tempat belanja favorit saran aja buat tokopedi...
17476,apabila aplikasi update baru berat jalan fungs...
17477,tampil baru sekarang sangat bagustapi rincian ...


In [80]:
df.drop_duplicates(subset='komentar',inplace=True)
df.shape

(16470, 1)

In [81]:
df

,komentar
0,dulu mikir pembeli ternyata sebagai penjual ...
1,udah banyak banget korban sama kalau salah la...
2,saran sebaiknya ambil langkah hukum melaporka...
3,bantu viralin admin kagak beres ini banyak be...
4,marketplace berani putus kerja sama bareng jn...
...,...
17474,tadi pernah kasi bintang 5 update akhir buat t...
17475,tempat belanja favorit saran aja buat tokopedi...
17476,apabila aplikasi update baru berat jalan fungs...
17477,tampil baru sekarang sangat bagustapi rincian ...


In [36]:
# Menghapus kolom yang diinginkan
df = df.drop(columns='userName')
df.head(20)

,komentar
0,baru puji pelayananya bagus eh aplikasi nya ma...
1,kebiasaan tokopedia habis pembayaran selalu a...
2,belanja tokopedia menyenangkan diskon harga mu...
3,cs nya semakin manusiawi aplikasi bodoh semua ...
4,ambil foto paylater diulang terus keterangan b...
5,kasih bintang 5 deh perbaiki dong masa dh men...
6,gak jelas lama-lama sistem tokopedia kupon pro...
7,aku ada masalah sama barang aku bermasalah sam...
8,banyakin diskonnya pengembalian barang sangat ...
9,gak bebas ongkir via cod payah keranjang gak m...


**Save into new Dataset**

In [82]:
df.to_csv('/Users/alipmaskhuri/skripsi/Dataset/finaldata.csv', index=False, encoding='utf-8')

menggabungkan dataset

In [17]:
# Baca kedua file CSV
df1 = pd.read_csv('/Users/alipmaskhuri/skripsi/Dataset/xkotor.csv')
df2 = pd.read_csv('/Users/alipmaskhuri/skripsi/Dataset/playsore_kotor.csv')

# Gabungkan kedua DataFrame
df_gabungan = pd.concat([df1, df2], ignore_index=True)

# Simpan hasil penggabungan ke file CSV baru
df_gabungan.to_csv('/Users/alipmaskhuri/skripsi/Dataset/dataset.csv', index=False)

In [ ]:
# Baca kedua file CSV
df1 = pd.read_csv('/Users/alipmaskhuri/skripsi/Dataset/tokoped_x_clean.csv')
df2 = pd.read_csv('/Users/alipmaskhuri/skripsi/tokped_gstore_clean.csv')

# Gabungkan kedua DataFrame
df_gabungan = pd.concat([df1, df2], ignore_index=True)

# Simpan hasil penggabungan ke file CSV baru
df_gabungan.to_csv('/Users/alipmaskhuri/skripsi/Dataset/tokpedcleanfix.csv', index=False)

In [86]:
file_path = "/Users/alipmaskhuri/skripsi/Dataset/tokpedcleanfix.csv"
df = pd.read_csv(file_path)

In [87]:
df.shape

(16464, 1)